<a href="https://colab.research.google.com/github/swalehaparvin/HuggingFace/blob/main/Hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install huggingface_hub
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

inputs = tokenizer("Hello, I like america", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]


'POSITIVE'

In [14]:
from transformers import pipeline
!pip install huggingface_hub
text = "AI-powered robots assist in complex brain surgeries with precision."

# Create the pipeline
classifier = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli")

# Create the categories list
categories = ["politics", "science", "sports"]

# Predict the output
output = classifier(text,categories)

# Print the top label and its score
print(f"Top Label: {output['labels'][0]} with score: {output['scores'][0]}")

Device set to use cpu


Top Label: science with score: 0.9510334134101868


Calculate and display the dataset size in megabytes

In [15]:
# Import the function to load dataset metadata
from datasets import load_dataset_builder
# Initialize the dataset builder for the MMLU-Pro dataset and Display dataset metadata
reviews_builder = load_dataset_builder("TIGER-Lab/MMLU-Pro")
print(reviews_builder.info)
# Calculate and print the dataset size in MB
dataset_size_mb = reviews_builder.info.dataset_size / (1024 ** 2)
print(f"Dataset size: {round(dataset_size_mb, 2)} MB")


DatasetInfo(description='', citation='', homepage='', license='', features={'question_id': Value(dtype='int64', id=None), 'question': Value(dtype='string', id=None), 'options': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'answer': Value(dtype='string', id=None), 'answer_index': Value(dtype='int64', id=None), 'cot_content': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None), 'src': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='mmlu-pro', config_name='default', version=0.0.0, splits={'validation': SplitInfo(name='validation', num_bytes=61143, num_examples=70, shard_lengths=None, dataset_name=None), 'test': SplitInfo(name='test', num_bytes=8715262, num_examples=12032, shard_lengths=None, dataset_name=None)}, download_checksums=None, download_size=71204160, post_processing_size=None, dataset_size=8776405, size_in_bytes=None)
Dataset size: 8.37 MB


# Manipulating datasets
There will likely be many occasions when you will need to manipulate a dataset before using it within a ML task. Two common manipulations are filtering and selecting (or slicing). The dataset is already loaded for you under wikipedia.
Filter the dataset for rows with the term "football" in the text column and save as filtered.
Select a single example from the filtered dataset and save as example.

In [ ]:
from datasets import load_dataset
wikipedia = load_dataset("wikipedia", "20220301.en", split="train")
# Filter the documents
filtered = wikipedia.filter(lambda row: "football" in row["text"])
# Create a sample dataset
example = filtered.select(range(1))
print(example[0]["text"])

# Grammatical correctness
Text classification is the process of labeling an input text into a pre-defined category. This can take the form of sentiment - positive or negative - spam detection - spam or not spam - and even grammatical errors.
Explore the use of a text-classification pipeline for checking an input sentence for grammatical errors.


In [ ]:
# Create a pipeline for grammar checking
grammar_checker = pipeline(
  task="text-classification",
  model="abdulmatinomotoso/English_Grammar_Checker"
)
# Check grammar of the input text
output = grammar_checker("I will walk dog")
print(output)

# Question Natural Language Inference
Another task under the text classification umbrella is Question Natural Language Inference, or QNLI. This checks if a premise contains enough information to answer a posed question, determining whether the answer can be found in the given text.
Create a text classification QNLI pipeline using the model "cross-encoder/qnli-electra-base" and save as classifier.
Use this classifier to determine if the text provides enough information to answer the question.

In [17]:
# Create the pipeline
classifier=pipeline(task="text-classification", model="cross-encoder/qnli-electra-base")
# Predict the output
output = classifier("Where is the capital of France?, Brittany is known for its stunning coastline.")
print(output)

Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.016211949288845062}]


# Dynamic category assignment
Dynamic category assignment enables a model to classify text into predefined categories, even without prior training for those categories.
Build the pipeline and save as a classifier.
Create a list of the labels - "politics", "science", "sports" - and save as categories.
Predict the label of text using the classifier and predefined categories.

In [18]:
text = "AI-powered robots assist in complex brain surgeries with precision."
# Create the pipeline
classifier = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli")
# Create the categories list
categories = ["politics", "science", "sports"]
# Predict the output
output = classifier(text,categories)
# Print the top label and its score
print(f"Top Label: {output['labels'][0]} with score: {output['scores'][0]}")

Device set to use cpu


Top Label: science with score: 0.9510334134101868


# Summarizing long text
Summarization reduces large text into manageable content, helping readers quickly grasp key points from lengthy articles or documents.
There are two main types: extractive, which selects key sentences from the original text, and abstractive, which generates new sentences summarizing main ideas.

In [20]:
# Create the summarization pipeline
summarizer = pipeline(task="summarization", model="cnicu/t5-small-booksum")
# Summarize the text
summary_text = summarizer(text)
# Compare the length
print(f"Original text length: {len(text)}")
print(f"Summary length: {len(summary_text[0]['summary_text'])}")

Device set to use cpu
Your max_length is set to 200, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


Original text length: 67
Summary length: 166


# Using min_length and max_length
The pipeline() function, has two important parameters: min_length and max_length. These are useful for adjusting the length of the resulting summary text to be short, longer, or within a certain number of words.

In [22]:
# Create a short summarizer
short_summarizer = pipeline(task="summarization", model="cnicu/t5-small-booksum", min_length=1, max_length=10)
# Summarize the input text
short_summary_text = short_summarizer(text)
# Print the short summary
print(short_summary_text[0]["summary_text"])
# Repeat for a long summarizer
long_summarizer = pipeline(task="summarization", model="cnicu/t5-small-booksum", min_length=50, max_length=150)
long_summary_text = long_summarizer(text)
# Print the long summary
print(long_summary_text[0]["summary_text"])

Device set to use cpu


AI-powered robots assist in complex brain


Device set to use cpu
Your max_length is set to 150, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


AI-powered robots assist in complex brain surgeries with precision . robots are powered by AI-based robots that are able to perform complex operations with precision and precision in a complex brain operation, allowing them to perform a variety of tasks.
